In [1]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        29/10/2025  -  23:00:15           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    30/10/2025  -  02:55:40           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================#



# Gatos vs Perretes 

idea de diseño: 
- **EfficientNet-B3 entrenado DESDE CERO** (sin pesos preentrenados de ImageNet)
- **K-Fold con validación cruzada** (5 folds) para mejor generalización
- **Entrenamiento por etapas**: primero solo la cabeza, luego fine-tuning completo
- **Data Augmentation** con Albumentations
- **Mixed Precision Training** para acelerar el entrenamiento
- **Learning rate alto** (adaptado para entrenamiento desde cero)

In [2]:
# Importo las librerías
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# PyTorch y movidas varias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# modelos preentrenados con transfer
import timm

# Data augmentation y transformaciones
import albumentations as A
from albumentations.pytorch import ToTensorV2

# K-Fold validación cruzada 
from sklearn.model_selection import StratifiedKFold

# fijo todas las semillas
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Para usar las graficas de kagle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


## 1. Configuración Global
**[v2.0 - 31/10/2025 03:00 AM]** - ADAPTADO para entrenamiento desde cero (sin preentrenado)

Hiperparámetros ajustados para entrenar EfficientNet-B3 completamente desde cero:
- **Epochs aumentados** (30+50 = 80 total)
- **Learning rates 10x mayores** (pesos aleatorios necesitan más actualización)
- **Batch size aumentado** (mejor convergencia)
- **Weight decay reducido** (no frenar aprendizaje inicial)

In [ ]:
# Configuración de parámetros y rutas
# Detecto automáticamente si estamos en Kaggle o en local
import os

if os.path.exists('/kaggle/input'):
    # Estamos en Kaggle
    BASE_PATH = '/kaggle/input/u-tad-dogs-vs-cats-2025'
    print("Entorno detectado: KAGGLE")
else:
    # Estamos en local
    BASE_PATH = '/home/nirmata/Descargas/u-tad-dogs-vs-cats-2025'
    print("Entorno detectado: LOCAL")

print(f"Ruta base del dataset: {BASE_PATH}\n")

CONFIG = {
    'train_dir': os.path.join(BASE_PATH, 'train/train'),
    'test_dir': os.path.join(BASE_PATH, 'test/test'),
    'supplementary_dir': os.path.join(BASE_PATH, 'supplementary_data/supplementary_data'),
    
    'model_name': 'efficientnet_b3',
    'img_size': 224,  # Reducido para entrenar más rápido
    'num_classes': 2,
    
    'batch_size': 64,  # Aumentado para mejor convergencia
    'num_folds': 5,
    'epochs_stage1': 30,  # Aumentado significativamente (sin pesos preentrenados)
    'epochs_stage2': 50,  # Aumentado para fine-tuning completo
    'lr_stage1': 1e-2,    # 10x mayor (pesos aleatorios necesitan más actualización)
    'lr_stage2': 1e-3,    # 10x mayor
    'weight_decay': 1e-4, # Reducido para no frenar el aprendizaje inicial
    'label_smoothing': 0.1,
    
    'num_workers': 2,
    'seed': 42
}

print("Configuración cargada:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

Entorno detectado: LOCAL
Ruta base del dataset: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025

Configuración cargada:
   train_dir: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025/train/train
   test_dir: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025/test/test
   supplementary_dir: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025/supplementary_data/supplementary_data
   model_name: efficientnet_b3
   img_size: 300
   num_classes: 2
   batch_size: 32
   num_folds: 5
   epochs_stage1: 5
   epochs_stage2: 15
   lr_stage1: 0.001
   lr_stage2: 0.0001
   weight_decay: 0.01
   label_smoothing: 0.1
   num_workers: 2
   seed: 42


In [4]:
# ============================================================================
# DIAGNÓSTICO: Verificar estructura del dataset
# ============================================================================

import os

print("="*60)
print("DIAGNOSTICO DE ESTRUCTURA DEL DATASET")
print("="*60)

# Verifico la ruta base
if os.path.exists(BASE_PATH):
    print(f"\n[OK] Ruta base encontrada: {BASE_PATH}")
    
    # Listo contenido principal
    print(f"\nContenido de la ruta base:")
    for item in os.listdir(BASE_PATH):
        item_path = os.path.join(BASE_PATH, item)
        if os.path.isdir(item_path):
            print(f"  [DIR]  {item}/")
        else:
            print(f"  [FILE] {item}")
    
    # Verifico directorio de entrenamiento
    print(f"\nVerificando train_dir: {CONFIG['train_dir']}")
    if os.path.exists(CONFIG['train_dir']):
        print(f"  [OK] Directorio existe")
        train_contents = os.listdir(CONFIG['train_dir'])
        print(f"  Contenido: {train_contents}")
        
        # Si hay subcarpetas, cuento imágenes
        for subdir in train_contents:
            subdir_path = os.path.join(CONFIG['train_dir'], subdir)
            if os.path.isdir(subdir_path):
                jpg_count = len([f for f in os.listdir(subdir_path) if f.endswith('.jpg')])
                print(f"    {subdir}/: {jpg_count} imágenes")
    else:
        print(f"  [ERROR] Directorio no existe")
    
    # Verifico directorio de test
    print(f"\nVerificando test_dir: {CONFIG['test_dir']}")
    if os.path.exists(CONFIG['test_dir']):
        print(f"  [OK] Directorio existe")
        test_count = len([f for f in os.listdir(CONFIG['test_dir']) if f.endswith('.jpg')])
        print(f"  Total de imágenes de test: {test_count}")
    else:
        print(f"  [ERROR] Directorio no existe")

else:
    print(f"\n[ERROR] Ruta base no encontrada: {BASE_PATH}")
    print("\nSoluciones:")
    if os.path.exists('/kaggle/input'):
        print("  - En Kaggle: Agrega el dataset como input")
    else:
        print("  - En local: Descarga el dataset y actualiza BASE_PATH")

print("\n" + "="*60)

DIAGNOSTICO DE ESTRUCTURA DEL DATASET

[OK] Ruta base encontrada: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025

Contenido de la ruta base:
  [DIR]  test/
  [DIR]  train/
  [DIR]  supplementary_data/
  [FILE] sample_submission.csv

Verificando train_dir: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025/train/train
  [OK] Directorio existe
  Contenido: ['Dog', 'Cat']
    Dog/: 12500 imágenes
    Cat/: 12500 imágenes

Verificando test_dir: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025/test/test
  [OK] Directorio existe
  Total de imágenes de test: 1067



## Preparación del Dataset

Creo un dataset personalizado de PyTorch y preparo los datos para validación cruzada K-Fold.

In [5]:
# Dataset personalizado
class DogsVsCatsDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        image = np.array(image)
        
        if self.transforms:
            image = self.transforms(image=image)['image']
        
        label = self.labels[idx]
        return image, label

# Preparo los datos de entrenamiento
def prepare_data(train_dir):
    """
    Carga las rutas de las imágenes y sus etiquetas desde el directorio de entrenamiento.
    Soporta dos estructuras:
    1. Imágenes directamente en train_dir (cat.123.jpg, dog.456.jpg)
    2. Imágenes en subcarpetas Cat/ y Dog/
    
    Args:
        train_dir: Ruta al directorio con las imágenes de entrenamiento
        
    Returns:
        image_paths: Array numpy con las rutas completas a las imágenes
        labels: Array numpy con las etiquetas (0=gato, 1=perro)
    """
    # Verifico que el directorio existe
    if not os.path.exists(train_dir):
        raise FileNotFoundError(
            f"ERROR: El directorio de entrenamiento no existe: {train_dir}\n"
            f"   Por favor, verifica que:\n"
            f"   1. Estás ejecutando en Kaggle (si usas paths de Kaggle)\n"
            f"   2. O actualiza CONFIG['train_dir'] con la ruta local correcta\n"
            f"   3. O descarga el dataset en la ubicación esperada"
        )
    
    print(f"Leyendo imágenes desde: {train_dir}")
    
    image_paths = []
    labels = []
    
    # Listo todos los archivos del directorio
    all_files = os.listdir(train_dir)
    
    # Verifico si hay subcarpetas Cat/ y Dog/
    has_subdirs = any(item in all_files for item in ['Cat', 'Dog', 'cat', 'dog'])
    
    if has_subdirs:
        print("   Estructura detectada: Subcarpetas Cat/ y Dog/")
        
        # Busco en subcarpetas
        for subdir in ['Cat', 'Dog', 'cat', 'dog']:
            subdir_path = os.path.join(train_dir, subdir)
            if not os.path.exists(subdir_path):
                continue
            
            # Determino la etiqueta según el nombre de la subcarpeta
            label = 0 if subdir.lower() == 'cat' else 1
            class_name = "gato" if label == 0 else "perro"
            
            # Listo las imágenes en la subcarpeta
            jpg_files = [f for f in os.listdir(subdir_path) if f.endswith('.jpg')]
            
            print(f"   {subdir}/: {len(jpg_files)} imágenes ({class_name})")
            
            for filename in jpg_files:
                filepath = os.path.join(subdir_path, filename)
                image_paths.append(filepath)
                labels.append(label)
    
    else:
        print("   Estructura detectada: Imágenes directas (cat.*.jpg, dog.*.jpg)")
        
        # Busco imágenes directamente en el directorio
        jpg_files = [f for f in all_files if f.endswith('.jpg')]
        print(f"   Archivos .jpg encontrados: {len(jpg_files)}")
        
        for filename in jpg_files:
            filepath = os.path.join(train_dir, filename)
            image_paths.append(filepath)
            
            # Etiqueto según el nombre del archivo
            if filename.startswith('cat'):
                label = 0
            elif filename.startswith('dog'):
                label = 1
            else:
                print(f"   [!] Archivo ignorado (nombre no reconocido): {filename}")
                continue
            
            labels.append(label)
    
    # Verifico que se encontraron imágenes
    if len(image_paths) == 0:
        raise ValueError(
            f"ERROR: No se encontraron imágenes .jpg válidas en: {train_dir}\n"
            f"   Contenido encontrado: {all_files}\n"
            f"   Verifica que el dataset está descargado correctamente"
        )
    
    # Convierto a arrays numpy
    image_paths = np.array(image_paths)
    labels = np.array(labels)
    
    return image_paths, labels

# Cargo los datos
print("\n" + "="*60)
print("CARGANDO DATOS DE ENTRENAMIENTO")
print("="*60)

train_paths, train_labels = prepare_data(CONFIG['train_dir'])

print(f"\n[OK] Datos cargados correctamente:")
print(f"   Total de imágenes: {len(train_paths)}")
print(f"   Gatos (label=0): {(train_labels == 0).sum()}")
print(f"   Perros (label=1): {(train_labels == 1).sum()}")
print(f"   Forma de train_paths: {train_paths.shape}")
print(f"   Forma de train_labels: {train_labels.shape}")
print("="*60 + "\n")


CARGANDO DATOS DE ENTRENAMIENTO
Leyendo imágenes desde: /home/nirmata/Descargas/u-tad-dogs-vs-cats-2025/train/train
   Estructura detectada: Subcarpetas Cat/ y Dog/
   Cat/: 12500 imágenes (gato)
   Dog/: 12500 imágenes (perro)

[OK] Datos cargados correctamente:
   Total de imágenes: 25000
   Gatos (label=0): 12500
   Perros (label=1): 12500
   Forma de train_paths: (25000,)
   Forma de train_labels: (25000,)



## Transformaciones y Data Augmentation
**[v1.5 - 30/10/2025 02:20 AM]** - Augmentation mejorado con ShiftScaleRotate

Defino las transformaciones de entrenamiento con augmentation y las de validación/test sin augmentation.

In [6]:
# Transformaciones de entrenamiento con augmentation
train_transforms = A.Compose([
    A.RandomResizedCrop(height=CONFIG['img_size'], width=CONFIG['img_size'], scale=(0.8, 1.0)),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# Transformaciones de validación/test sin augmentation
val_transforms = A.Compose([
    A.Resize(height=CONFIG['img_size'], width=CONFIG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

print("Transformaciones definidas")
print(f"   - Train: RandomCrop, HorizontalFlip, Brightness/Contrast, Rotation")
print(f"   - Val/Test: Solo resize y normalización")

ValueError: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.8, 1.0), 'ra...': 1.0, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

## Modelo con EfficientNet-B3 (ENTRENADO DESDE CERO)
**[v2.0 - 31/10/2025 03:00 AM]** - CRÍTICO: Cambiado a entrenamiento desde cero (sin pesos preentrenados)

⚠️ **IMPORTANTE**: Este modelo se entrena completamente desde cero, sin usar pesos de ImageNet.
Todos los parámetros se inicializan aleatoriamente y se aprenden durante el entrenamiento.

In [ ]:
# Creo modelo con transfer learning (PERO SIN PESOS PREENTRENADOS)
def create_model(model_name, num_classes, pretrained=False):  # CAMBIADO A FALSE
    model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)
    print(f"   [INFO] Modelo creado DESDE CERO (pretrained={pretrained})")
    print(f"   [INFO] Todos los pesos inicializados aleatoriamente")
    return model

# Congelo el backbone para etapa 1
def freeze_backbone(model):
    for name, param in model.named_parameters():
        if 'classifier' not in name:
            param.requires_grad = False
    return model

# Descongelo todo para etapa 2
def unfreeze_backbone(model):
    for param in model.parameters():
        param.requires_grad = True
    return model

model = create_model(CONFIG['model_name'], CONFIG['num_classes'], pretrained=False)  # CAMBIADO A FALSE
model = model.to(device)

print(f"Modelo creado: {CONFIG['model_name']}")
print(f"   - Parámetros totales: {sum(p.numel() for p in model.parameters()):,}")
print(f"   - Parámetros entrenables: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## Funciones de Entrenamiento y Validación
**[v1.3 - 30/10/2025 00:45 AM]** - Añadido Mixed Precision Training (AMP)

Implemento las funciones para entrenar y validar el modelo con mixed precision training.

In [ ]:
# Función de entrenamiento con mixed precision
def train_epoch(model, dataloader, criterion, optimizer, scaler, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

# Función de validación
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

print("Funciones de entrenamiento y validación definidas")

## Entrenamiento con K-Fold Cross-Validation

commits Hechos:

**[v1.1 - 30/10/2025 00:10 AM]** - Implementado K-Fold (5 folds)  
**[v1.2 - 30/10/2025 00:30 AM]** - Añadido Early Stopping y Label Smoothing

Entreno el modelo con 5 folds y 2 etapas por fold:
1. **Etapa 1**: Solo entreno la cabeza con el backbone congelado
2. **Etapa 2**: Fine-tuning completo descongelando todo el modelo

In [ ]:
# ============================================================================
# VERIFICACIÓN PRE-ENTRENAMIENTO
# ============================================================================
print("\n" + "="*60)
print("VERIFICACION DE DATOS ANTES DEL ENTRENAMIENTO")
print("="*60)

# 1. Verifico que las variables existen y no están vacías
print("\n[1] Verificando variables train_paths y train_labels:")
try:
    print(f"   [OK] train_paths existe: {type(train_paths)}")
    print(f"   [OK] train_labels existe: {type(train_labels)}")
    print(f"   [OK] Número de muestras: {len(train_paths)}")
    print(f"   [OK] Tipos correctos: {train_paths.dtype}, {train_labels.dtype}")
except NameError as e:
    print(f"   [ERROR] Variable no definida - {e}")
    raise

# 2. Verifico que las longitudes coinciden
print("\n[2] Verificando que las listas son paralelas:")
if len(train_paths) == len(train_labels):
    print(f"   [OK] Longitudes coinciden: {len(train_paths)} == {len(train_labels)}")
else:
    raise ValueError(f"   [ERROR] Longitudes no coinciden: {len(train_paths)} != {len(train_labels)}")

# 3. Verifico distribución de clases
print("\n[3] Verificando distribución de clases:")
unique_labels, counts = np.unique(train_labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    class_name = "Gato" if label == 0 else "Perro"
    percentage = (count / len(train_labels)) * 100
    print(f"   Clase {label} ({class_name}): {count} muestras ({percentage:.2f}%)")

# 4. Verifico que las rutas de archivos existen
print("\n[4] Verificando existencia de archivos (muestreo de 5):")
sample_indices = np.random.choice(len(train_paths), size=min(5, len(train_paths)), replace=False)
for idx in sample_indices:
    path = train_paths[idx]
    exists = os.path.exists(path)
    label = train_labels[idx]
    status = "[OK]" if exists else "[ERROR]"
    print(f"   {status} {os.path.basename(path)} (label={label})")

# 5. Verifico que puedo cargar una imagen de muestra
print("\n[5] Probando carga de una imagen de muestra:")
try:
    test_img_path = train_paths[0]
    test_img = Image.open(test_img_path).convert('RGB')
    test_img_array = np.array(test_img)
    print(f"   [OK] Imagen cargada correctamente")
    print(f"   Dimensiones: {test_img_array.shape}")
    print(f"   Rango de valores: [{test_img_array.min()}, {test_img_array.max()}]")
except Exception as e:
    print(f"   [ERROR] al cargar imagen: {e}")
    raise

# 6. Verifico compatibilidad con StratifiedKFold
print("\n[6] Verificando compatibilidad con StratifiedKFold:")
try:
    test_skf = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])
    splits = list(test_skf.split(train_paths, train_labels))
    print(f"   [OK] StratifiedKFold generó {len(splits)} splits correctamente")
    print(f"   Tamaños de los splits:")
    for i, (train_idx, val_idx) in enumerate(splits[:3]):  # Muestro solo los primeros 3
        print(f"      Fold {i+1}: Train={len(train_idx)}, Val={len(val_idx)}")
except Exception as e:
    print(f"   [ERROR] en StratifiedKFold: {e}")
    raise

print("\n" + "="*60)
print("[OK] TODAS LAS VERIFICACIONES PASADAS - LISTO PARA ENTRENAR")
print("="*60 + "\n")

In [ ]:
# K-Fold cross-validation
skf = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])

fold_models = []
fold_metrics = []

print(f"Iniciando entrenamiento con {CONFIG['num_folds']} folds\n")

for fold, (train_idx, val_idx) in enumerate(skf.split(train_paths, train_labels)):
    print(f"{'='*60}")
    print(f"FOLD {fold + 1}/{CONFIG['num_folds']}")
    print(f"{'='*60}")
    
    fold_train_paths = train_paths[train_idx]
    fold_train_labels = train_labels[train_idx]
    fold_val_paths = train_paths[val_idx]
    fold_val_labels = train_labels[val_idx]
    
    train_dataset = DogsVsCatsDataset(fold_train_paths, fold_train_labels, transforms=train_transforms)
    val_dataset = DogsVsCatsDataset(fold_val_paths, fold_val_labels, transforms=val_transforms)
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], 
                             shuffle=True, num_workers=CONFIG['num_workers'], pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], 
                           shuffle=False, num_workers=CONFIG['num_workers'], pin_memory=True)
    
    model = create_model(CONFIG['model_name'], CONFIG['num_classes'], pretrained=False)  # DESDE CERO
    model = model.to(device)
    
    # ETAPA 1: entreno solo la cabeza
    print(f"\nEtapa 1: Entrenando solo la cabeza ({CONFIG['epochs_stage1']} epochs)")
    model = freeze_backbone(model)
    
    criterion = nn.CrossEntropyLoss(label_smoothing=CONFIG['label_smoothing'])
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                           lr=CONFIG['lr_stage1'], weight_decay=CONFIG['weight_decay'])
    scaler = GradScaler()
    
    best_val_acc = 0.0
    
    for epoch in range(CONFIG['epochs_stage1']):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{CONFIG['epochs_stage1']} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
    
    # ETAPA 2: fine-tuning completo
    print(f"\nEtapa 2: Fine-tuning completo ({CONFIG['epochs_stage2']} epochs)")
    model = unfreeze_backbone(model)
    
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr_stage2'], 
                           weight_decay=CONFIG['weight_decay'])
    
    best_val_acc = 0.0
    patience_counter = 0
    patience_limit = 5
    
    for epoch in range(CONFIG['epochs_stage2']):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{CONFIG['epochs_stage2']} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            print(f"   -> Nuevo mejor modelo (Val Acc: {val_acc:.2f}%)")
        else:
            patience_counter += 1
        
        if patience_counter >= patience_limit:
            print(f"   -> Early stopping en epoch {epoch+1}")
            break
    
    model.load_state_dict(best_model_state)
    
    fold_models.append(model)
    fold_metrics.append({
        'fold': fold + 1,
        'best_val_acc': best_val_acc,
        'val_loss': val_loss
    })
    
    print(f"\nFold {fold + 1} completado - Mejor Val Acc: {best_val_acc:.2f}%\n")

# Resumen de todos los folds
print(f"\n{'='*60}")
print("RESUMEN DE ENTRENAMIENTO")
print(f"{'='*60}")
for metric in fold_metrics:
    print(f"Fold {metric['fold']}: Val Acc = {metric['best_val_acc']:.2f}%")

avg_val_acc = np.mean([m['best_val_acc'] for m in fold_metrics])
print(f"\nAccuracy promedio en validación: {avg_val_acc:.2f}%")
print(f"{'='*60}\n")

## Inferencia en Test Set

Cargo las imágenes de test y genero predicciones promediando los 5 modelos de los folds.

In [ ]:
# Dataset de test
class TestDataset(Dataset):
    def __init__(self, test_dir, transforms=None):
        self.test_dir = test_dir
        self.transforms = transforms
        self.image_files = sorted([f for f in os.listdir(test_dir) if f.endswith('.jpg')])
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.test_dir, img_name)
        
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        
        if self.transforms:
            image = self.transforms(image=image)['image']
        
        img_id = int(img_name.split('.')[0])
        return image, img_id

test_dataset = TestDataset(CONFIG['test_dir'], transforms=val_transforms)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], 
                         shuffle=False, num_workers=CONFIG['num_workers'], pin_memory=True)

print(f"Dataset de test cargado: {len(test_dataset)} imágenes")

In [ ]:
# Genero predicciones promediando los 5 folds
print("Generando predicciones...")

all_predictions = []
all_ids = []

for model in fold_models:
    model.eval()

with torch.no_grad():
    for images, img_ids in test_loader:
        images = images.to(device)
        
        fold_preds = []
        for model in fold_models:
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            fold_preds.append(probs.cpu().numpy())
        
        avg_probs = np.mean(fold_preds, axis=0)
        all_predictions.extend(avg_probs)
        all_ids.extend(img_ids.numpy())

print(f"Predicciones generadas: {len(all_predictions)} imágenes")

## Generación del archivo Submission

Creo el archivo `submission.csv` con las predicciones finales para subir a Kaggle.

In [ ]:
# Creo el submission
# Convierto las probabilidades a etiquetas binarias (0=gato, 1=perro)
binary_predictions = (np.array(all_predictions) > 0.5).astype(int)

submission_df = pd.DataFrame({
    'id': all_ids,
    'label': binary_predictions
})

submission_df = submission_df.sort_values('id').reset_index(drop=True)
submission_df.to_csv('submission.csv', index=False)

print("Archivo submission.csv generado correctamente")
print(f"\nPrimeras predicciones:")
print(submission_df.head(10))
print(f"\nDistribución de predicciones:")
print(f"   - Gatos (label=0): {(submission_df['label'] == 0).sum()}")
print(f"   - Perros (label=1): {(submission_df['label'] == 1).sum()}")
print(f"\nTotal de predicciones: {len(submission_df)}")

## Visualización de Predicciones (mi bonus)

Muestro algunas imágenes del test set con sus predicciones para verificar visualmente.

In [ ]:
# Visualizo predicciones aleatorias
def visualize_predictions(num_images=9):
    random_indices = np.random.choice(len(submission_df), size=num_images, replace=False)
    
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.ravel()
    
    for i, idx in enumerate(random_indices):
        img_id = submission_df.iloc[idx]['id']
        prediction = submission_df.iloc[idx]['label']
        
        img_path = os.path.join(CONFIG['test_dir'], f"{img_id}.jpg")
        img = Image.open(img_path)
        
        predicted_class = "Perro" if prediction > 0.5 else "Gato"
        confidence = prediction if prediction > 0.5 else 1 - prediction
        
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(f"ID: {img_id}\n{predicted_class} ({confidence*100:.1f}%)", 
                         fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_predictions(num_images=9)